# Import libraries

In [ ]:
#IMPORT NECESSARY LIBRARIES AND ESTABLISH LINK WITH FILES
import keras
keras.__version__
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

import matplotlib.pyplot as plt

import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
import re

np.set_printoptions(threshold=np.inf) #Prevents array truncation

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Making the data usable

In [ ]:
header=['NAME','TEAM','AGE','GP','W','L','MIN','OFFRTG','DEFRTG','NETRTG','AST%','AST/TO','AST Ratio','OREB%','DREB%','REB%','TO Ratio','eFG%','TS%','USG%','PACE','PI']
col = {}
for i in np.arange(len(header)):
  col[header[i]]=i

target_header=['TEAM','SEASON','AGAINST TEAM','WIN/LOSS']
target_column = {}
for i in np.arange(len(target_header)):
  target_column[target_header[i]]=i
  
    
os.chdir("/content/drive/My Drive/sports")

In [ ]:
winfile = ['sports 96-97.txt', 'sports 97-98.txt', 'sports 98-99.txt', 'sports 99-00.txt', 'sports 00-01.txt', 
           'sports 01-02.txt', 'sports 02-03.txt', 'sports 03-04.txt', 'sports 04-05.txt', 'sports 05-06.txt',
           'sports 06-07.txt', 'sports 07-08.txt', 'sports 08-09.txt', 'sports 09-10.txt', 'sports 10-11.txt',
           'sports 11-12.txt', 'sports 12-13.txt', 'sports 13-14.txt', 'sports 14-15.txt', 'sports 15-16.txt',
           'sports 16-17.txt', 'sports 17-18.txt', 'sports 18-19.txt', 'sports 19-20.txt']

final_tensor = np.zeros((24,541,22))

fspecial = re.sub('\n','\t',open('Target Data Points by team.txt').read()).split('\t')[4:]

for q in range(len(winfile)):
  
  infile = winfile[q]
  text = open(infile).read()
  oneline = re.sub('\n', '\t', text)
  f = oneline.split('\t')


  pnum = {}
  tnum = {}
  team = {}
  name = []

  cols = len(header)
  rows = int(len(f)/(len(header)))

  stats = np.zeros((rows+5,cols))

  teami=1
  for i in np.arange(rows):
    n = i*(cols)
    newname = f[n]
    if f[n+1] not in tnum:
      tnum[f[n+1]] = teami
      teami += 1
    f[n+1] = tnum[f[n+1]]
    if newname not in pnum:
      name.append(newname)
      pnum[name[i]] = i
      f[n] = pnum[f[n]]
      team[name[i]] = f[n+1]
      for j in np.arange(cols):
        stats[i,j] = f[n+j]
  for i in range(len(fspecial)//4):
    n = i*4
    if int(fspecial[n+1]) == q+1:
      fspecial[n] = tnum[fspecial[n]]
      fspecial[n+2] = tnum[fspecial[n+2]]
  for i in range(cols):
    for j in range(rows):
      final_tensor[q,j,i]=stats[j,i]

target_tensor=np.zeros(len(fspecial))
for i in range(len(fspecial)):
  target_tensor[i] = fspecial[i]
target_tensor = target_tensor.reshape((len(fspecial)//4,4))

In [ ]:
print(final_tensor[:,:,1])
#ngame = 0
#nsea = target_column[ngame,1]
#team = target_column[game,0]
#home = final_tensor[nsea,final_tensor[nsea,:,1]==team,2:22]

# Rescaling data

In [ ]:
######Rescaling data
#################################################################################
##########First 11 seasons are training data, next 5 are validation, next 8 are test data
#WE MUST MEAN-SUBTRACT AND STD-DIVIDE BY THE MEAN AND STD OF THE ENTIRE TRAINING SET
for n in range(2,22):
  final_tensor_slice = final_tensor[:,:,n]
  final_tensor_slice = final_tensor_slice.reshape(541*24)
  new_list = []
  for i in range(541*11):
    if final_tensor_slice[i] != int(0):
      new_list.append(final_tensor_slice[i])
  avg = np.mean(new_list)
  std = np.std(new_list)

  for i in range(len(final_tensor_slice)):
    if abs(final_tensor_slice[i]) != int(0):
      final_tensor_slice[i] = (final_tensor_slice[i]-avg)/std

In [ ]:
infile = 'Target Data Points by team.txt'
text = open(infile).read()
#oneline = re.sub('\t', '', text)
f = text.split('\n')
f = f[1:]

hteam = []
ateam = []
nsea = []
labels = []
for i in np.arange(len(f)):
  z = f[i].split('\t')
  hteam.append(z[0])
  ateam.append(z[2])
  nsea.append(int(z[1]))
  labels.append(z[3])


In [ ]:
maxplayers = 25
home = []
away =[]
inmat = np.zeros((len(labels),maxplayers,20,2))
for ig in np.arange(len(labels)):
  if hteam[ig] in tnum and ateam[ig] in tnum:
    hnum = tnum[hteam[ig]]
    anum = tnum[ateam[ig]]
    home = final_tensor[nsea[ig]-1,final_tensor[nsea[ig]-1,:,1]==hnum,2:22]
    away = final_tensor[nsea[ig]-1,final_tensor[nsea[ig]-1,:,1]==anum,2:22]
    nhplay = np.shape(home)[0]
    naplay = np.shape(away)[0]
    inmat[ig,0:nhplay,:,0] = home
    inmat[ig,0:naplay,:,1] = away

In [ ]:
labels = keras.utils.to_categorical(labels)
print(np.shape(labels))

In [ ]:
#Make the training and testing sets
train_data = inmat[:13875,:,:,:]
train_targets = labels[:13875,:]
val_data = inmat[13875:20660,:,:,:]
val_targets = labels[13875:20660,:]
test_data = inmat[20660:,:,:]
test_targets = labels[20660:,:]
#NEED TO FIGURE OUT WHAT FROM THE TARGET TENSOR IS NECESSARY
#premature


In [ ]:
print(np.shape(train_data))

In [ ]:
print(final_tensor[:,:,1])

# In case you want to test that it worked

In [ ]:
##96-97
query = 0
print(final_tensor[query,:,:])

In [ ]:
##97-98
query = 1
print(final_tensor[query,:,:])

In [ ]:
##98-99
query = 2
print(final_tensor[query,:,:])

In [ ]:
##99-00
query = 3
print(final_tensor[query,:,:])

In [ ]:
##00-01
query = 4
print(final_tensor[query,:,:])

In [ ]:
##01-02
query = 5
print(final_tensor[query,:,:])

In [ ]:
#02-03
query = 6
print(final_tensor[query,:,:])

In [ ]:
#03-04
query = 7
print(final_tensor[query,:,:])

In [ ]:
#04-05
query = 8
print(final_tensor[query,:,:])

In [ ]:
#05-06
query = 9
print(final_tensor[query,:,:])

In [ ]:
#06-07
query = 10
print(final_tensor[query,:,:])

In [ ]:
#07-08
query = 11
print(final_tensor[query,:,:])

In [ ]:
#08-09
query = 12
print(final_tensor[query,:,:])

In [ ]:
#09-10
query = 13
print(final_tensor[query,:,:])

In [ ]:
#10-11
query = 14
print(final_tensor[query,:,:])

In [ ]:
#11-12
query = 15
print(final_tensor[query,:,:])

In [ ]:
#12-13
query = 16
print(final_tensor[query,:,:])

In [ ]:
#13-14
query = 17
print(final_tensor[query,:,:])

In [ ]:
#14-15
query = 18
print(final_tensor[query,:,:])

In [ ]:
#15-16
query = 19
print(final_tensor[query,:,:])

In [ ]:
#16-17
query = 20
print(final_tensor[query,:,:])

In [ ]:
#17-18
query = 21
print(final_tensor[query,:,:])

In [ ]:
#18-19
query = 22
print(final_tensor[query,:,:])

In [ ]:
##18-19
query = 23
print(final_tensor[query,:,:])

In [ ]:
print(final_tensor.shape)

# This is the part where we get the model together

In [ ]:
model = models.Sequential()
model.add(layers.Dense(64, activation='relu',
                       input_shape=(maxplayers,20,2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])


num_epochs = 50  # For each partition, run 200 epochs to train.
all_mae_histories = [] # Save each history, so we can see how it does

history = model.fit(train_data,
                    train_targets,
                    epochs=20,
                    validation_data=(val_data, val_targets))

history_dict = history.history
history_dict.keys()
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

In [ ]:
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'ro', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'bx-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()





plt.clf()   # clear figure
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']

plt.plot(epochs, acc, 'ro', label='Training acc')
plt.plot(epochs, val_acc, 'bx-', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Copy the code here that trains the model, but instead of going all the way out to 20 epochs, stop at the epoch you think makes the most sense.
best_epoch = 5 # Change this!!!
model.fit(train_data, train_targets, epochs=best_epoch)


In [ ]:
results = model.evaluate(test_data, test_targets)
print(results)

In [ ]:
predictions = model.predict(x_test)
print(np.shape(predictions))
print('Review\tLabel\tPrediction')
for i in np.arange(30):
  print(i, '\t',test_labels[i],'\t',predictions[i]) 



In [ ]:
#FROM THE BOSTON HOUSING MODEL EXAMPLE

def build_model():
    # Because we will need to instantiate
    # the same model multiple times,
    # we use a function to construct it.
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',
                           input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model






num_epochs = 200  # For each partition, run 200 epochs to train.
all_mae_histories = [] # Save each history, so we can see how it does

k = 4 # Use four partitions
num_val_samples = len(train_data) // k # Divide by k and round down

for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)




average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]




plt.plot(average_mae_history)
#plt.plot(average_mae_history[10:])
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()






# Get a fresh, compiled model.
max_epoch = 48
model = build_model()
# Train it on the entirety of the data.
model.fit(train_data, train_targets,
          epochs=max_epoch, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)





test_mae_score






predictions = model.predict(test_data)







plt.figure(figsize=(18,5))
plt.plot(predictions[:,0],'r.',label='Predictions')
plt.plot(test_targets,'bx',label='Targets')
plt.ylabel('Cost of House (k$)')
plt.legend()
plt.show()

plt.figure(figsize=(18,5))
resid = predictions[:,0]-test_targets
plt.plot(resid,'g+')
plt.ylabel('Diff. btw. Prediction and Actual')
plt.show()

plt.hist(resid,bins=20)
plt.xlabel('Diff. btw. Prediction and Actual')
plt.ylabel('Number of Cases')
plt.show()






new_mae = []  # Set up an empty array to keep the results

for i in np.arange(13): # Loop through the 13 inputs
  permutation = np.copy(test_data) # Make a copy of the test data
  permutation[:,i] = 0.0 # Set a particular column of inputs to zero
  p_mse, p_mae = model.evaluate(permutation, test_targets) # evaluate the modified data
  new_mae.append(p_mae-test_mae_score) # save the change in MAE to the array

plt.bar(np.arange(13)+1,new_mae) # Make a bar graph to show change in result
plt.ylabel('Change in MAE')
plt.xlabel('Parameter Number (see list above)')
plt.show()








num_epochs = 200  # For each partition, run 200 epochs to train.
all_mae_histories = [] # Save each history, so we can see how it does

k = 5 # Use four partitions
num_val_samples = len(train_data) // k # Divide by k and round down

for i in range(k):
    print('processing fold #', i)
    # Prepare the validation data: data from partition # k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]

    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = build_model()
    # Train the model (in silent mode, verbose=0)
    history = model.fit(partial_train_data, partial_train_targets,
                        validation_data=(val_data, val_targets),
                        epochs=num_epochs, batch_size=1, verbose=0)
    mae_history = history.history['val_mae']
    all_mae_histories.append(mae_history)






average_mae_history = [np.mean([x[i] for x in all_mae_histories]) for i in range(num_epochs)]







plt.plot(average_mae_history)
#plt.plot(average_mae_history[10:])
plt.xlabel('Epochs')
plt.ylabel('Validation MAE')
plt.show()